In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
def load_and_preprocess_data(file_path, window_size=50, step=10):
    # Load your data - adjust this based on your data format
    data = pd.read_csv(file_path)
    
    # Separate features (X) and labels (y)
    X = data[['x', 'y', 'z']].values
    y = data['status'].values
    
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Create windowed data
    X_windowed = []
    y_windowed = []
    for i in range(0, len(X) - window_size + 1, step):
        X_windowed.append(X[i:i+window_size])
        y_windowed.append(y[i+window_size-1])
    
    X_windowed = np.array(X_windowed)
    y_windowed = np.array(y_windowed)
    
    # Convert labels to categorical
    y_windowed = to_categorical(y_windowed)
    
    return X_windowed, y_windowed

# Create the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Train the model
def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1)
    return history

if __name__ == "__main__":
    # Load and preprocess data
    X, y = load_and_preprocess_data('M02_Feb_2021_OP13_000.csv')
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create and train the model
    input_shape = X_train.shape[1:]
    num_classes = y_train.shape[1]
    model = create_cnn_model(input_shape, num_classes)
    
    history = train_model(model, X_train, y_train, X_test, y_test)
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {test_accuracy:.4f}")
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    
    print("Example predictions:")
    for i in range(5):
        print(f"Sample {i+1}: Predicted class: {predicted_classes[i]}, Actual class: {np.argmax(y_test[i])}")

c:\Users\Abhyuday Patel\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9261 - loss: 0.2453 - val_accuracy: 0.9927 - val_loss: 0.0494
Epoch 2/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9879 - loss: 0.0549 - val_accuracy: 0.9953 - val_loss: 0.0300
Epoch 3/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9931 - loss: 0.0395 - val_accuracy: 0.9948 - val_loss: 0.0333
Epoch 4/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9925 - loss: 0.0480 - val_accuracy: 0.9933 - val_loss: 0.0386
Epoch 5/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9955 - loss: 0.0372 - val_accuracy: 0.9948 - val_loss: 0.0277
Epoch 6/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9952 - loss: 0.0269 - val_accuracy: 0.9948 - val_loss: 0.0338
Epoch 7/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9928 - loss: 0.0369 - val_accuracy: 0.9948 - val_loss: 0.0243
Epoch 8/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9956 - loss: 0.0276 - val_acc

In [2]:
from sklearn.metrics import r2_score

# Assuming y_true is your true values and y_pred is your cNN model predictions
r_squared = r2_score(y_test, predictions)
r_squared

0.9898553705082271

In [10]:
model.predict(X_test)[:,1]

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([1.000000e+00, 1.000000e+00, 1.000000e+00, ..., 1.000000e+00,
       1.000000e+00, 1.065697e-23], dtype=float32)